In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from math import floor
from scipy.stats import gaussian_kde

from utility.types import RegimeDetectionModels
from data.universe import Universe
from data.benchmark import Benchmark
from portfolio_management.market_regime import detect_market_regime
from portfolio_management.beta_estimation import (
    estimate_dynamic_beta_and_alpha,
    historical_rolling_beta,
)

# Data loading


In [2]:
bench = Benchmark()
benchmark = bench.get_benchmark_returns_data()
benchmark, bench.benchmark_perf

(             OISESTR       SPX      SX5T
 2016-05-19  0.000000  0.000000  0.000000
 2016-05-20  0.000000  0.000000  0.000000
 2016-05-23  0.000000  0.000000  0.000000
 2016-05-24  0.000000  0.000000  0.000000
 2016-05-25  0.000000  0.000000  0.000000
 ...              ...       ...       ...
 2024-03-01  0.000000  0.005785  0.001428
 2024-03-04  0.000000  0.000539  0.004018
 2024-03-05  0.000392 -0.008862 -0.004131
 2024-03-06  0.000196  0.001269  0.005315
 2024-03-07  0.000196  0.003952  0.012250
 
 [2036 rows x 3 columns],
 2016-05-19    1.000000
 2016-05-20    1.000000
 2016-05-23    1.000000
 2016-05-24    1.000000
 2016-05-25    1.000000
                 ...   
 2024-03-01    1.658112
 2024-03-04    1.661620
 2024-03-05    1.655421
 2024-03-06    1.660339
 2024-03-07    1.671945
 Freq: B, Name: benchmark_perf, Length: 2036, dtype: float64)

In [3]:
universe = Universe()
universe_data = universe.get_universe_price()
universe_data

,EUROPE _VALUE_FACTOR,EUROPE _MOMENTUM_FACTOR,WATER_ESG,STOXX_EUROPE 600_TECHNOLOGY,STOXX_EUROPE 600_HEALTHCARE,SX5T_levier_2,NASDAQ-100_LEVIER_2,Px fut SX5E,Px fut sp500,Px fut nasdaq,SX5T,SPTR500N,ESTR_ETF
2016-05-24,183.5100,50.499,34.320,34.870,82.230,18.465,95.935,2999.0,2075.00,4445.25,46.485,112.10,100.016
2016-05-25,186.5900,50.725,34.570,35.380,83.560,19.130,97.640,3049.0,2087.25,4474.75,46.485,112.10,100.016
2016-05-26,187.1000,50.907,34.600,35.465,83.550,19.260,98.000,3061.0,2089.75,4492.25,46.485,112.10,100.016
2016-05-27,187.1800,51.078,34.790,35.700,84.180,19.375,99.565,3065.0,2097.25,4510.00,46.485,112.10,100.016
2016-05-30,187.6100,51.213,34.760,35.765,84.310,19.520,100.110,3065.0,2097.25,4510.00,46.485,112.10,100.016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,282.0773,97.850,63.441,98.682,143.874,51.830,976.900,4901.0,5146.00,18338.25,77.150,333.84,101.980
2024-03-04,281.1000,98.090,63.670,99.413,144.528,52.180,983.400,4918.0,5138.25,18262.00,77.460,334.02,101.980
2024-03-05,281.8439,97.920,63.290,97.776,143.792,51.760,946.200,4897.0,5085.75,17930.25,77.140,331.06,102.020
2024-03-06,282.8483,98.330,63.310,98.998,144.003,52.300,955.400,4922.0,5111.75,18044.25,77.550,331.48,102.040


In [4]:
universe_data = pd.merge(
    universe_data, bench.benchmark_perf, left_index=True, right_index=True
).pct_change().fillna(0)
universe_data

,EUROPE _VALUE_FACTOR,EUROPE _MOMENTUM_FACTOR,WATER_ESG,STOXX_EUROPE 600_TECHNOLOGY,STOXX_EUROPE 600_HEALTHCARE,SX5T_levier_2,NASDAQ-100_LEVIER_2,Px fut SX5E,Px fut sp500,Px fut nasdaq,SX5T,SPTR500N,ESTR_ETF,benchmark_perf
2016-05-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2016-05-25,0.016784,0.004475,0.007284,0.014626,0.016174,0.036014,0.017772,0.016672,0.005904,0.006636,0.000000,0.000000,0.000000,0.000000
2016-05-26,0.002733,0.003588,0.000868,0.002402,-0.000120,0.006796,0.003687,0.003936,0.001198,0.003911,0.000000,0.000000,0.000000,0.000000
2016-05-27,0.000428,0.003359,0.005491,0.006626,0.007540,0.005971,0.015969,0.001307,0.003589,0.003951,0.000000,0.000000,0.000000,0.000000
2016-05-30,0.002297,0.002643,-0.000862,0.001821,0.001544,0.007484,0.005474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,0.004568,0.004620,0.005994,0.015707,0.013283,0.006408,0.026371,0.001430,0.008278,0.014129,0.001428,0.005785,0.000000,0.001873
2024-03-04,-0.003465,0.002453,0.003610,0.007408,0.004546,0.006753,0.006654,0.003469,-0.001506,-0.004158,0.004018,0.000539,0.000000,0.002116
2024-03-05,0.002646,-0.001733,-0.005968,-0.016467,-0.005092,-0.008049,-0.037828,-0.004270,-0.010217,-0.018166,-0.004131,-0.008862,0.000392,-0.003731
2024-03-06,0.003564,0.004187,0.000316,0.012498,0.001467,0.010433,0.009723,0.005105,0.005112,0.006358,0.005315,0.001269,0.000196,0.002971


# Regime detection


In [5]:
REGIMES = detect_market_regime(
    market_data=universe_data[["benchmark_perf"]],#.pct_change().fillna(0).to_numpy(),
    market_regime_detection_algorithm=RegimeDetectionModels.HIDDEN_MARKOV_MODEL,
    scale_data=True,
    scaler_type="robust",
)
universe_data["REGIMES"] = REGIMES
universe_data

,EUROPE _VALUE_FACTOR,EUROPE _MOMENTUM_FACTOR,WATER_ESG,STOXX_EUROPE 600_TECHNOLOGY,STOXX_EUROPE 600_HEALTHCARE,SX5T_levier_2,NASDAQ-100_LEVIER_2,Px fut SX5E,Px fut sp500,Px fut nasdaq,SX5T,SPTR500N,ESTR_ETF,benchmark_perf,REGIMES
2016-05-24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2016-05-25,0.016784,0.004475,0.007284,0.014626,0.016174,0.036014,0.017772,0.016672,0.005904,0.006636,0.000000,0.000000,0.000000,0.000000,0
2016-05-26,0.002733,0.003588,0.000868,0.002402,-0.000120,0.006796,0.003687,0.003936,0.001198,0.003911,0.000000,0.000000,0.000000,0.000000,0
2016-05-27,0.000428,0.003359,0.005491,0.006626,0.007540,0.005971,0.015969,0.001307,0.003589,0.003951,0.000000,0.000000,0.000000,0.000000,0
2016-05-30,0.002297,0.002643,-0.000862,0.001821,0.001544,0.007484,0.005474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,0.004568,0.004620,0.005994,0.015707,0.013283,0.006408,0.026371,0.001430,0.008278,0.014129,0.001428,0.005785,0.000000,0.001873,0
2024-03-04,-0.003465,0.002453,0.003610,0.007408,0.004546,0.006753,0.006654,0.003469,-0.001506,-0.004158,0.004018,0.000539,0.000000,0.002116,0
2024-03-05,0.002646,-0.001733,-0.005968,-0.016467,-0.005092,-0.008049,-0.037828,-0.004270,-0.010217,-0.018166,-0.004131,-0.008862,0.000392,-0.003731,0
2024-03-06,0.003564,0.004187,0.000316,0.012498,0.001467,0.010433,0.009723,0.005105,0.005112,0.006358,0.005315,0.001269,0.000196,0.002971,0


In [6]:
universe_data[universe_data['REGIMES']==0].mean().sort_values()

REGIMES                        0.000000
ESTR_ETF                       0.000017
STOXX_EUROPE 600_HEALTHCARE    0.000556
benchmark_perf                 0.000727
EUROPE _VALUE_FACTOR           0.000863
EUROPE _MOMENTUM_FACTOR        0.000937
WATER_ESG                      0.000938
Px fut SX5E                    0.000955
SX5T                           0.000977
Px fut sp500                   0.001056
SPTR500N                       0.001166
Px fut nasdaq                  0.001419
STOXX_EUROPE 600_TECHNOLOGY    0.001459
SX5T_levier_2                  0.002068
NASDAQ-100_LEVIER_2            0.002909
dtype: float64

In [7]:
universe_data[universe_data['REGIMES']==1].mean().sort_values()

SX5T_levier_2                 -0.002858
NASDAQ-100_LEVIER_2           -0.002804
STOXX_EUROPE 600_TECHNOLOGY   -0.001767
SX5T                          -0.001546
Px fut SX5E                   -0.001509
EUROPE _VALUE_FACTOR          -0.001410
WATER_ESG                     -0.001323
EUROPE _MOMENTUM_FACTOR       -0.001206
SPTR500N                      -0.001083
Px fut sp500                  -0.001047
benchmark_perf                -0.001009
Px fut nasdaq                 -0.001002
STOXX_EUROPE 600_HEALTHCARE   -0.000301
ESTR_ETF                      -0.000010
REGIMES                        1.000000
dtype: float64

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(25, 15))
for i, sec in enumerate(["MONTAIRE", "ETF_GROWTH", "ETF_VALUE", "ETF_CAC", "ETF_SPX"]):
    ax_l = ax[i].twinx()
    ax_l.fill_between(
        universe_data.index,
        universe_data["REGIMES"],
        alpha=0.2,
        color="red",
        step="pre",
    )
    ax[i].plot(universe_data[sec], label=f"{sec} returns")
    ax[i].set_xlabel("Datetime", fontsize=15)
    ax[i].set_ylabel("Returns", fontsize=15)
    ax_l.set_ylabel("REGIMES", fontsize=15)
    ax[i].set_title(f"Regimes on {sec}", fontsize=20)
    ax[i].grid()
    ax[i].legend(fontsize=15)

# Dynamic beta estimation


In [ ]:
BETA_TO_OBSERVE = "ETF_VALUE"

In [ ]:
estimated_alpha, estimated_beta = estimate_dynamic_beta_and_alpha(
    universe_data["benchmark_perf"].pct_change().fillna(0).to_numpy(),
    universe_data[BETA_TO_OBSERVE].pct_change().fillna(0).to_numpy(),
)

historical_OLS_beta = historical_rolling_beta(
    universe_data["benchmark_perf"].pct_change().fillna(0).to_numpy(),
    universe_data[BETA_TO_OBSERVE].pct_change().fillna(0).to_numpy(),
    50,
)

In [ ]:
universe_data["BETA_GROWTH_ETF_TO_BENCH"] = estimated_beta

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 10))
ax_l = ax.twinx()
ax_l.plot(
    universe_data["BETA_GROWTH_ETF_TO_BENCH"],
    label="Dynamic betas",
    color="orange",
)
ax_l.plot(
    universe_data.index,
    historical_OLS_beta,
    label="Dynamic betas benchmark (rolling OLS)",
    color="green",
    linestyle="--",
)

ax_l.legend(loc="upper right", fontsize=15)
ax.plot(
    (universe_data["benchmark_perf"].pct_change().fillna(0) + 1).cumprod(),
    label="SPX returns",
    color="blue",
    alpha=0.8,
)
ax.plot(
    (universe_data[BETA_TO_OBSERVE].pct_change().fillna(0) + 1).cumprod(),
    label=f"{BETA_TO_OBSERVE} returns",
    color="red",
    alpha=0.8,
)
ax.set_xlabel("Datetime", fontsize=15)
ax.set_ylabel("Returns", fontsize=15)
ax_l.set_ylabel("Beta", fontsize=15)
ax.set_title("Dynamic beta", fontsize=20)
# ax.set_yscale('log')
ax.grid()
ax.legend(fontsize=15)

# Beta analysis among regime


In [ ]:
universe_data_returns = universe_data.copy()
for col in [
    "MONTAIRE",
    "ETF_GROWTH",
    "ETF_VALUE",
    "ETF_CAC",
    "ETF_SPX",
    "benchmark_perf",
]:
    universe_data_returns[col] = universe_data_returns[col].pct_change().fillna(0)

universe_data_returns

In [ ]:
universe_data_returns.groupby(by=["REGIMES"]).mean()

In [ ]:
universe_data_returns

In [ ]:

fig, ax = plt.subplots(
    round(
        universe_data_returns.drop(columns=["REGIMES"]).columns.shape[0]
        / 2
    ),
    2,
    figsize=(25, 25),
)
k = 0
for i, sec in enumerate(universe_data_returns.drop(columns=["REGIMES"]).columns):
    samples_1 = sorted(
        universe_data_returns[universe_data_returns["REGIMES"] == 1][sec].to_numpy()
    )
    samples_0 = sorted(
        universe_data_returns[universe_data_returns["REGIMES"] == 0][sec].to_numpy()
    )
    if i % 2 != 0 :
        c = 1
    else:
        c = 0

    ax[floor(i/2), c].hist(
        samples_0,
        bins=50,
        density=True,
        alpha=0.4,
        color="blue",
        label="Bullish regime",
    )
    ax[floor(i/2), c].plot(
        samples_0,
        gaussian_kde(samples_0, bw_method="scott").pdf(samples_0),
        color="blue",
        label="Bearish regime KDE",
    )
    ax[floor(i/2), c].hist(
        samples_1,
        bins=50,
        density=True,
        alpha=0.4,
        color="orange",
        label="Bearish regime",
    )
    ax[floor(i/2), c].plot(
        samples_1,
        gaussian_kde(samples_1, bw_method="scott").pdf(samples_1),
        color="orange",
        label="Bearish regime KDE",
    )

    ax[floor(i/2), c].set_xlabel(f"{sec}", fontsize=10)
    ax[floor(i/2), c].set_ylabel("Density", fontsize=12)
    ax[floor(i/2), c].set_title(f"{sec} density by regime", fontsize=15)
    ax[floor(i/2), c].grid()
    ax[floor(i/2), c].legend(fontsize=12)


# Beta regime


In [ ]:
universe_data

In [ ]:
universe_data["BETA_REGIMES"] = detect_market_regime(
    market_data=universe_data[["BETA_GROWTH_ETF_TO_BENCH"]]
    .fillna(0)
    .to_numpy(),
    market_regime_detection_algorithm=RegimeDetectionModels.HIDDEN_MARKOV_MODEL,
    scale_data=True,
    scaler_type="robust",
)

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(25, 25))
for i, sec in enumerate(["MONTAIRE", "ETF_GROWTH", "ETF_VALUE", "ETF_CAC", "ETF_SPX"]):
    ax_l = ax[i].twinx()
    ax_l.fill_between(
        universe_data.index,
        universe_data["BETA_REGIMES"],
        alpha=0.2,
        color="red",
        step="pre",
    )
    ax[i].plot(universe_data[sec], label=f"{sec} returns")
    ax[i].set_xlabel("Datetime", fontsize=10)
    ax[i].set_ylabel("Returns", fontsize=12)
    ax_l.set_ylabel("REGIMES", fontsize=12)
    ax[i].set_title(f"Regimes on {sec}", fontsize=15)
    ax[i].grid()
    ax[i].legend(fontsize=12)